In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


In [2]:
def safe_request_loop(url, headers):
    attempt = 0
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print('Nessun errore')
                return response
            else:
                print(f"Errore {response.status_code} - ritento...")
        except Exception as e:
            print(f"Eccezione nella richiesta: {e} - ritento...")

        # Attendi prima di riprovare
        wait_time = min(60, 2 ** attempt + random.uniform(0, 2))
        print(f"Aspetto {wait_time:.2f} secondi prima del tentativo successivo...")
        time.sleep(wait_time)
        attempt += 1

In [29]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": "https://www.google.com/"
}
url = 'https://www.transfermarkt.it/serie-a/spieltag/wettbewerb/IT1/plus/?saison_id=2024&spieltag=2'
response = requests.get(url, headers=headers)

In [38]:

soup = BeautifulSoup(response.content, 'html.parser')

section = soup.find('div', class_='large-8 columns')

boxes = section.find_all("div", class_ = "box")


In [74]:
match_data = []
goal_data = []

for box in boxes:
    # Salta box che non contengono risultati
    if not box.find("span", class_="matchresult"):
        continue

    try:
        # Squadre e risultato
        home_team_box = box.find("td", class_ = 'rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname')
        #home_team = home_team_box.find("a", href = True).text.strip()

        if home_team_box:
            for a in home_team_box.find_all("a", href=True):
                if a.get("title") != "Discutere della giornata" and a.text.strip():
                    home_team = a.text.strip()
                    break
        away_team_box = box.find("td", class_ = 'hauptlink no-border-links no-border-rechts hide-for-small spieltagsansicht-vereinsname')
        away_team = away_team_box.find("a", href = True).text.strip()

        result = box.find("span", class_="matchresult").text.strip()

        match_data.append({
            "Squadra Casa": home_team,
            "Squadra Trasferta": away_team,
            "Risultato Finale": result
        })

        # Estrai tutti i goal
        events = box.find_all("tr", class_="spieltagsansicht-aktionen")
        
        for event in events:
            tds = event.find_all("td", class_ = "no-border spieltagsansicht-aktionen")
            
            #if len(tds) < 5:
            #    continue
            for td in tds:
                player_td = td.find('td', class_ = "spieltagsansicht")
                player_tag = player_td.find("span", class_ ="hide-for-small")
                player = player_tag.text.strip() 
                print(player)
                goal_result = td.find('td', class_ ="zentriert hauptlink").text.strip()
                #minutes = td.find('td', )
                
            minute = tds[1].text.strip().replace("'", "")
            #goal_result = tds[2].text.strip()
            scorer_tag = event.find("a", href=True)
            if not scorer_tag:
                continue
            scorer = scorer_tag.text.strip()

            if tds[0].text.strip():
                scoring_team = home_team
            else:
                scoring_team = away_team

            goal_data.append({
                "Giocatore": player,
                "Minuto": int(minute),
                "Squadra": scoring_team,
                "Risultato Goal": goal_result
            })
    except Exception as e:
        print(f"Errore durante l'elaborazione di una partita: {e}")

# Crea i DataFrame
df_partite = pd.DataFrame(match_data)
df_goal = pd.DataFrame(goal_data)

print("=== Partite ===")
print(df_partite)
print("\n=== Gol ===")
print(df_goal)

Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
Errore durante l'elaborazione di una partita: list index out of range
=== Partite ===
  Squadra Casa Squadra Trasferta Risultato Finale
0        Parma             Milan              2:1
1      Udinese             Lazio              2:1
2        Inter             Lecce              2:0
3        Monza             Genoa              0:1
4   Fiorentina           Venezia              0:0
5       Torino          Atalanta              2:1
6   

In [71]:
df_partite

,Squadra Casa,Squadra Trasferta,Risultato Finale
0,Parma,Milan,2:1
1,Udinese,Lazio,2:1
2,Inter,Lecce,2:0
3,Monza,Genoa,0:1
4,Fiorentina,Venezia,0:0
5,Torino,Atalanta,2:1
6,Napoli,Bologna,3:0
7,Roma,Empoli,1:2
8,Cagliari,Como,1:1
9,Verona,Juventus,0:3


In [43]:
boxes[1]

<div class="box" style="border-top: 0 !important;">
<table style="border-top: 0 !important;">
<tbody>
<tr class="table-grosse-schrift">
<td class="rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname">
<span class="tabellenplatz">(12.)</span> 
                                                                    <a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio">Parma</a> </td>
<td class="rechts hauptlink no-border-rechts show-for-small spieltagsansicht-vereinsname">
<span class="tabellenplatz">(12.)</span> 
                                                                    <a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio">Parma</a> </td>
<td class="hauptlink zentriert no-border-links no-border-rechts hide-for-small spieltagsansicht-wappen">
<a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio"><img alt="Parma Calcio" class="" src="https://tmssl.akamaized.net//